# Example 6.1 Logical Identity Group
This notebook illustrates the operation of the algorithm for determining the generators of the Logical Identity group.

## Diagonal Logical Identity Generators $\mathbf{M}_Z$

The algorithm assumes we are given the Codewords as input. 

We form $E_M$ from the Z-support of the code words, $E$, and a column of all ones. 

We then calculate the Howell basis $K$ of the kernel of $E_M$.

To run different scenarios click here: [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/m-webster/XPFpackage/HEAD?urlpath=tree/Examples)

This code is Copyright 2021-22 Mark Webster, and is made available subject to [GPL licensing](https://www.gnu.org/licenses/gpl-3.0.en.html).

In [1]:
import add_parent_dir
import numpy as np
from common import *
from NSpace import *
from XPAlgebra import *
from XPCodes import *

## Code 1
genstr = 'XP_8(8|0000000|6554444),XP_8(7|1111111|1241234),XP_8(1|1110000|3134444)'

## Code 2
# genstr = 'XP8(0|0000000|1322224)\nXP8(12|1111111|1234567)'


G, N = str2XP(genstr)
n = XPn(G)
print('The Code Generators are G:')
print(XP2Str(G,N),"\n")
C = Code(G,N)
Codewords = getVal(C,'Codewords')
print('The code words are:')
for i in range(len(Codewords)):
    print(f'|k_{i}> = {State2Str(Codewords[i],N)}')
print('\n')
E = np.vstack([XPx(c) for c in Codewords])
print('E_M is the ZSupport of the code words, E, plus a column of all ones:')
EM = np.hstack([E,np.ones((len(E),1),dtype=int)])
print(ZmatPrint(EM,2))
print('\n')
nsp = NSpace(EM,N)
nsp.simplifyKer()
K = nsp.K
print('K is the Howell basis of Ker(E_M):')
print(ZmatPrint(K,N))
print('\n')
print('The diagonal logical identity generators M_Z are then:')
MZ = Zp2XP(K,N,double=False)
print(XP2Str(MZ,N))

The Code Generators are G:
XP_8( 8|0000000|6554444)
XP_8( 7|1111111|1241234)
XP_8( 1|1110000|3134444) 

The code words are:
|k_0> = |0000001>+w6/16|0001110>+w9/16|1110001>+w15/16|1111110>
|k_1> = |0000010>+w4/16|0001101>+w9/16|1110010>+w13/16|1111101>
|k_2> = |0000100>+w2/16|0001011>+w9/16|1110100>+w11/16|1111011>
|k_3> = |0000111>+|0001000>+w9/16|1110111>+w9/16|1111000>


E_M is the ZSupport of the code words, E, plus a column of all ones:
00000011
11100011
00011101
11111101
00000101
11100101
00011011
11111011
00001001
11101001
00010111
11110111
00001111
11101111
00010001
11110001


K is the Howell basis of Ker(E_M):
10700000
01700000
00044444


The diagonal logical identity generators M_Z are then:
XP_8( 0|0000000|1070000)
XP_8( 0|0000000|0170000)
XP_8( 8|0000000|0004444)


## Non-Diagonal Logical Identity Generators $\mathbf{M}_X$
The process to find the non-diagonal generators is as follows:
1. Find the X components of the generators
2. Find the phase and Z components which are consistent with the relative phases between computational basis elements in code words


### Step 1: X-Components: 
Firstly, we find the X-components of the generators.  Let $E_i$ be the Z-support of the code word $|\kappa_i\rangle$. Take any element $\mathbf{e}_i \in E_i$ and let $T$ be the binary matrix with rows formed from  $\{ \mathbf{e}_i \oplus \mathbf{e}: \mathbf{e} \in E_i\}$. Let $S_X = \text{RREF}_{\mathbb{Z}_2}(T)$ and let $\mathbf{m}_i = \text{res}_{\mathbb{Z}_2}(T,\mathbf{e}_i)$. Verify that $E_i = \mathbf{m}_i + \langle S_X \rangle$ for each code word. If not, return FALSE.

In [2]:
i = 0
k_i = Codewords[i]
E_i = XPx(k_i)
e_i = E_i[0]
E_i_str = ZMat2str(E_i,2)
print(f'i={i}')
print(f'E_i is the Z-Support of codeword |k_{i}>:')
print("\n".join(E_i_str))
print('\n')
print('e_0 = E_i[0] is the first element of E_i:')
print(E_i_str[0])
print('\n')
print('We add e_0 to each element of E_i to form T = (E_i + e_i) mod 2:')
T = np.mod(E_i + e_i,2)
print(ZmatPrint(T,2))
print('\n')
# print('T is the set of all x in E_i\' such that e + x mod 2 in E_i for all e in E_i:')

# print(ZmatPrint(T,2))
# print('\n')
print('The X-components of the non-diagonal logical I generators are given by SXx = RREF(T):')
nsp2 = NSpace(T,2)
SXx = nsp2.H
print(ZmatPrint(SXx,2))

i=0
E_i is the Z-Support of codeword |k_0>:
0000001
1110001
0001110
1111110


e_0 = E_i[0] is the first element of E_i:
0000001


We add e_0 to each element of E_i to form T = (E_i + e_i) mod 2:
0000000
1110000
0001111
1111111


The X-components of the non-diagonal logical I generators are given by SXx = RREF(T):
1110000
0001111




### Step 2: Phase and Z-Components: 
Secondly, we use linear algebra modulo $N$ to find valid phase and Z-components for the generators. Assume that the codewords $|\kappa_i\rangle$ are written in orbit form as in Equation \ref{eq:orbitform}. For each $\mathbf{x}$ in $S_X$ we complete the following steps:
1. Let $\mathbf{e}'_{ij} = \mathbf{e}_{ij} \oplus \mathbf{x}$, and let $p'_{ij}$ be the phase of $\mathbf{e}'_{ij}$ in the code words
2. Let $p''_{ij} = (p'_{ij} - p_{ij}) \mod 2N$ - this is the change in phase when applying the operator $XP_N(0|\mathbf{x}|\mathbf{0})$
3. For there to be a valid solution, the $p''_{ij}$ are all either even or odd. Let $a = p_{00}'' \mod 2$ be an adjustment factor. Let $p'''_{ij} = (p''_{ij} - a)/2$ so that $p'''_{ij} \in \mathbb{Z}_N$. Let $\mathbf{p}'''$ be the vector formed from the $p'''_{ij}$
3. Find a solution $\mathbf{b} = (\mathbf{z}|q) \in \mathbb{Z}_N^{n}\times \mathbb{Z}_N$ such that $E_M \mathbf{b} = \mathbf{p}'''$ using linear algebra modulo $N$. 
4. If there is no such solution $\mathbf{b}$, return FALSE
5. Otherwise, let $M_Z = \{(\mathbf{z}_j|p_j)\}$ be the matrix formed from the Z and phase components of the operators in $\mathbf{M}_Z = \{XP_N(p_j|\mathbf{0}|\mathbf{z}_j)\}$. Let $(\mathbf{z}'|p') = \text{res}_{\mathbb{Z}_N}(M_Z,(\mathbf{z}|q))$ and add the operator $XP_N(a+2p'|\mathbf{x}|\mathbf{z}')$ to $\mathbf{M}_X$.

In [3]:
## Make a single codeword
CW = ZMat([np.vstack(Codewords)])

MX = []
CWphases = CW2Dict(CW)
setVerbose(True)
K2 = 2*K
for x in SXx:
    print('Finding a non-diagonal generator with X-component:',x)
    A = getLX(x,CW,CWphases,nsp,N)
    if A is not False:
        print('The resulting non-diagonal generator is:',XP2Str(A,N))
        Zp = XP2Zp(A,N)
        Zp,u = matResidual(K2,Zp,2*N)
        p,x0,z = XPcomponents(Zp2XP(Zp,N))
        A = makeXP(p,x,z)
        print('We update the phase and Z component by taking the residue with respect to K:',XP2Str(A,N))
        MX.append(A)
    print("\n")
setVerbose(False)        



Finding a non-diagonal generator with X-component: [1 1 1 0 0 0 0]
p''' represents the change in phase to each basis element in the codewords when applying the operator  XP_8( 0|1110000|0000000)
[4 3 4 3 4 3 4 3 4 3 4 3 4 3 4 3]
b = (z|q) represents the Z component and phase vector of an operator which applies the change in phase p''':
[7 0 0 4 4 4 4 0]
The resulting non-diagonal generator is: XP_8( 1|1110000|7004444)
We update the phase and Z component by taking the residue with respect to K: XP_8( 9|1110000|0070000)


Finding a non-diagonal generator with X-component: [0 0 0 1 1 1 1]
p''' represents the change in phase to each basis element in the codewords when applying the operator  XP_8( 0|0001111|0000000)
[3 3 5 5 2 2 6 6 1 1 7 7 0 0 0 0]
b = (z|q) represents the Z component and phase vector of an operator which applies the change in phase p''':
[0 0 0 1 2 3 4 7]
The resulting non-diagonal generator is: XP_8(14|0001111|0001234)
We update the phase and Z component by taking the re

We now show that each of stabilizer generator in G can be expressed in terms of the generators M_X, M_Z


In [4]:

def expressGroup(G,LI,N):
    for A in G:
        r, u = XPResidual(LI,A,N)
        temp = []
        temp.append(XP2Str(A,N))
        temp.append(" = ")
        for i in range(len(u)):
            a = u[i]
            B = LI[i]
            if a > 0:
                temp.append(XP2Str(B,N))
                if a > 1:
                    temp.append(f'^{a} ')
                else:
                    temp.append(' ')
        print("".join(temp),isZero(r))

G, N = str2XP(genstr)
n = XPn(G)
print('The Code Generators are: G =')
print(XP2Str(G,N),"\n")
C = Code(G,N)
LI = getVal(C,'LI')
print('The Logical Identity Group generators are: \nMX =')
print(XP2Str(MX,N))
print('MZ =')
print(XP2Str(MZ,N))
print('\nCheck that each element of G can be expressed as the product of elements of M. Hence <G> is a subgroup of <M>:')
expressGroup(G,LI,N)
print('\nNow check that each canonical generator can be expressed as the product of elements of M. Hence <S> is a subgroup of <M>:')
S = getVal(C,'S')
SX,SZ = splitDiag(S)
print("SX")
print(XP2Str(SX,N))
print("SZ")
print(XP2Str(SZ,N))
print("\n")
expressGroup(S,LI,N)

The Code Generators are: G =
XP_8( 8|0000000|6554444)
XP_8( 7|1111111|1241234)
XP_8( 1|1110000|3134444) 

The Logical Identity Group generators are: 
MX =
XP_8( 9|1110000|0070000)
XP_8(14|0001111|0001234)
MZ =
XP_8( 0|0000000|1070000)
XP_8( 0|0000000|0170000)
XP_8( 8|0000000|0004444)

Check that each element of G can be expressed as the product of elements of M. Hence <G> is a subgroup of <M>:
XP_8( 8|0000000|6554444) = XP_8( 0|0000000|1070000)^6 XP_8( 0|0000000|0170000)^5 XP_8( 8|0000000|0004444)  True
XP_8( 7|1111111|1241234) = XP_8( 9|1110000|0070000) XP_8(14|0001111|0001234) XP_8( 0|0000000|1070000)^7 XP_8( 0|0000000|0170000)^6  True
XP_8( 1|1110000|3134444) = XP_8( 9|1110000|0070000) XP_8( 0|0000000|1070000)^5 XP_8( 0|0000000|0170000)^7 XP_8( 8|0000000|0004444)  True

Now check that each canonical generator can be expressed as the product of elements of M. Hence <S> is a subgroup of <M>:
SX
XP_8( 9|1110000|1240000)
XP_8(14|0001111|0001234)
SZ
XP_8( 8|0000000|2334444)
XP_8( 0|00000

## Summary
When comparing the logical identity generators with the canonical generators, it is often the case that the group $\mathcal{I}_\text{XP} = \langle \mathbf{M}_X, \mathbf{M}_Z\rangle \ne \langle\mathbf{G}\rangle = \langle \mathbf{S}_X, \mathbf{S}_Z\rangle$. In general:
- The number of non-diagonal generators is always the same, with the same X-components but with some simplification to the phase and Z components. 
- There may be more diagonal generators  in $\mathbf{M}_Z$ compared to $\mathbf{S}_Z$ but $\mathbf{S}_Z \subset \langle \mathbf{M}_Z\rangle$.